In [38]:
!pip install pymupdf

In [39]:
import requests

import requests
import fitz  # PyMuPDF
from io import BytesIO

import nltk
nltk.download('punkt_tab')
from nltk.tokenize import word_tokenize
from collections import Counter
import string

[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!


In [40]:
smoking_query = {} # TODO: name this whatever for your particular query, make sure you replace future references to smoking_query with the new name

def get_papers(query, limit=100):
    url = "https://api.semanticscholar.org/graph/v1/paper/search"
    params = {
        "query": query,
        "limit": limit,
        "fields": "title,authors,year,url,citationCount,referenceCount"
    }
    response = requests.get(url, params=params)
    return response.json()["data"]

papers = get_papers("Effects of smoking on lung cancer") # TODO: Change this for respective query
for i, paper in enumerate(papers, 1):
  if i <= 30 or i > 70:
    authors = paper.get('authors', 'N/A')
    if authors != 'N/A':
      authors = [author['name'] for author in authors]
    smoking_query[paper["url"]] = {'title': paper.get("title", "N/A"), 'authors': authors, 'year': paper.get("year", 0),
                                   'citations': paper.get("citationCount", 0), 'references': paper.get("referenceCount", 0),
                                   'relevance': 2 if i <= 30 else 1}

In [41]:
smoking_query.keys() # first 30 are relevant documents (relevance = 2), last 30 are semi-relevant documents (relevance = 1)

dict_keys(['https://www.semanticscholar.org/paper/5cdc4a4dffe96963f6600c6bd422b2e0c8ae79c4', 'https://www.semanticscholar.org/paper/6ff89b62e1ffa490050db958abb43b32e1bacc24', 'https://www.semanticscholar.org/paper/10cf5ba57728f06062351c19197d530cf92711a6', 'https://www.semanticscholar.org/paper/288733edb6fc7b954bcb4fbf430e9b5ba20bc5f8', 'https://www.semanticscholar.org/paper/d1a4a81f2763d3a25a5ff4ca78713be79c60231b', 'https://www.semanticscholar.org/paper/4f4315c7b33831e545eaf84c0614c3fcf785203f', 'https://www.semanticscholar.org/paper/886b716a9e249b54f67defad0821eef798ae510a', 'https://www.semanticscholar.org/paper/268600086ce7dfa282fae6e363d8c2674c906aa8', 'https://www.semanticscholar.org/paper/2877b325151ddd6eea962166677597f062e56502', 'https://www.semanticscholar.org/paper/861af2586241e55a979e102db79acea9eaca2b11', 'https://www.semanticscholar.org/paper/af773fc6d81e3cc0f1fc7eb2bf3dd2f95645c9e5', 'https://www.semanticscholar.org/paper/e6cc6233fcb6800d4570c872159c64857e428fbe', 'http

In [42]:
def extract_text_from_pdf_url(pdf_urls):
  urls = list(smoking_query.keys())
  for i, pdf_url in enumerate(pdf_urls):
    response = requests.get(pdf_url)
    with fitz.open(stream=BytesIO(response.content), filetype="pdf") as doc:
      pdf_text = "\n".join([page.get_text() for page in doc])
    tokens = word_tokenize(pdf_text)
    tokens = [token for token in tokens if token not in string.punctuation]
    word_counts = Counter(tokens)
    smoking_query[urls[i]]['tokens'] = word_counts

# TODO
pdf_urls = [] # fill out as list of pdf urls (need to get them manually), should be of size 60 (make sure it's in the same order as the urls in the previous box)
extract_text_from_pdf_url(pdf_urls)

In [43]:
print(smoking_query[list(smoking_query.keys())[0]]) # can use this to check specific url values

{'title': 'Differential effects of smoking on lung cancer mortality before and after household stove improvement in Xuanwei, China', 'authors': ['Kyoung Mu Lee', 'R. Chapman', 'M. Shen', 'J. Lubin', 'D. Silverman', 'Xingzhou He', 'H. Hosgood', 'Bingshu E. Chen', 'P. Rajaraman', 'N. Caporaso', 'J. Fraumeni', 'A. Blair', 'Q. Lan'], 'year': 2010, 'citations': 22, 'references': 16, 'relevance': 2}


In [44]:
import pickle

# save query dict
with open("word_counts.pkl", "wb") as f:
    pickle.dump(smoking_query, f)

# load query dict
with open("word_counts.pkl", "rb") as f:
    my_data = pickle.load(f)
